## Description

This file will contain the analysis of brain recordings comparing resting state behavior with tapping activity. 
It will take the following steps:

1. Prepare environment
2. Import data (1 participant at a time)
3. Clean data:
    1. Apply filtering (0.5 - 50Hz)
    2. Apply Notch
    3. Remove flatlined channels
    4. Mark noisy channels
    5. Perform ICA
    6. Remove components that correlated with EOG and ECG
    7. Remove muscle components (spiky activity in 20-50Hz frequency range)
    8. Interpolate noisy channels
4. Re-reference data
5. Analyse data...

In [1]:
# Install required libraries

%pip install mne numpy matplotlib PyQt5 scikit-learn



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Perform required imports

import PyQt5
import mne
import matplotlib.pyplot as plt
import numpy as np
from mne.preprocessing import ICA
import sklearn
from mne.time_frequency import psd_array_welch
%matplotlib qt


In [85]:
# Define your files
resting_files = [ "participant_02_resting-epo.fif", "participant_03_resting-epo.fif"]
tapping_files = [ "participant_02_tapping-epo.fif", "participant_03_tapping-epo.fif"]

# Define channels to filter out
exclude_channels = ['BIP3', 'BIP7', 'BIP8', 'BIP9']

# Load epochs and harmonize channels
def harmonize_epochs(file_list, exclude_channels):
    epochs_list = []
    common_channels = None  # To store the intersection of channel names

    # Step 1: Load all epochs and find common channels
    for file in file_list:
        epochs = mne.read_epochs(file, preload=True)
        if common_channels is None:
            common_channels = set(epochs.info['ch_names'])
        else:
            common_channels &= set(epochs.info['ch_names'])
        epochs_list.append(epochs)

    # Retain only common channels
    common_channels = list(common_channels)
    print(f"Common channels across all files: {common_channels}")

    # Step 2: Drop excluded channels from common channels
    common_channels = [ch for ch in common_channels if ch not in exclude_channels]
    print(f"Channels after exclusion: {common_channels}")

    # Step 3: Apply common channel set to each epochs object
    print(common_channels)

    harmonized_epochs_list = []
    for epochs in epochs_list:
        epochs.pick_channels(common_channels)
        harmonized_epochs_list.append(epochs)

    return harmonized_epochs_list

# Process the files
resting_epochs_list = harmonize_epochs(resting_files, exclude_channels)
tapping_epochs_list = harmonize_epochs(tapping_files, exclude_channels)

# Combine epochs
group_resting_epochs = mne.concatenate_epochs(resting_epochs_list)
group_tapping_epochs = mne.concatenate_epochs(tapping_epochs_list)

print(f"Combined resting epochs shape: {group_resting_epochs.get_data().shape}")
print(f"Combined tapping epochs shape: {group_tapping_epochs.get_data().shape}")


Reading c:\Users\ymijs\OneDrive\Documents\Internship\Apathy-Agency\EEG Experiment\Data Analysis\participant_02_resting-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9999.00 ms
        0 CTF compensation matrices available
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\ymijs\OneDrive\Documents\Internship\Apathy-Agency\EEG Experiment\Data Analysis\participant_03_resting-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9999.00 ms
        0 CTF compensation matrices available
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Common channels across all files: ['Fp1', 'F4', 'T7', 'FC5', 'AF8', 'Fz', 'BIP9', 'C1', 'C4', 'CP4', 'C5', 'F7', 'C3', 'P7', 'CP6', 'FC2', 'CP2', 'P8', 'FC1', 'POz', 'PO5', 'T8', 'TP7', 'Fpz', 'PO4', 'AF3', 'F5', 'P5', 'CP5', 'Fp2', 'P1', 'PO3', 'PO7', 'FC6', 'P6', 'PO6', 'F1', 'FC3', 'B

C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\556710079.py:45: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  group_resting_epochs = mne.concatenate_epochs(resting_epochs_list)
C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\556710079.py:46: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  group_tapping_epochs = mne.concatenate_epochs(tapping_epochs_list)


Not setting metadata
317 matching events found
No baseline correction applied
Combined resting epochs shape: (60, 59, 10000)
Combined tapping epochs shape: (317, 59, 10001)


In [42]:
from mne.time_frequency import psd_array_welch
import matplotlib.pyplot as plt

# Compute PSD for each condition using Welch's method
resting_psd, resting_freqs = psd_array_welch(
    group_resting_epochs.get_data(),
    sfreq=group_resting_epochs.info['sfreq'],
    fmin=0.5,
    fmax=50,
    n_fft=256  # Length of FFT, adjust for frequency resolution
)

tapping_psd, tapping_freqs = psd_array_welch(
    group_tapping_epochs.get_data(),
    sfreq=group_tapping_epochs.info['sfreq'],
    fmin=0.5,
    fmax=50,
    n_fft=256
)

# Average PSD over channels (axis=1 corresponds to the channel dimension)
resting_psd_avg = resting_psd.mean(axis=1)
tapping_psd_avg = tapping_psd.mean(axis=1)

# Average PSD over channels (axis=1 corresponds to the channel dimension)
resting_psd_avg_avg = resting_psd_avg.mean(axis=0)
tapping_psd_avg_avg = tapping_psd_avg.mean(axis=0)

# Plot the group-level PSD comparison
plt.figure(figsize=(8, 6))
plt.plot(resting_freqs, resting_psd_avg.mean(axis=0), label="Resting", color="blue", linewidth=2)
plt.plot(tapping_freqs, tapping_psd_avg.mean(axis=0), label="Tapping", color="red", linewidth=2)
plt.legend(loc="upper right", fontsize=12)
plt.xlabel("Frequency (Hz)", fontsize=12)
plt.ylabel("Power Spectral Density", fontsize=12)
plt.title("Group-Level PSD Comparison: Resting vs. Tapping", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


In [39]:
# Plot in dB unnormalized (log scale)

# Step 5: Convert power to dB (log scale)
# Add a small constant to avoid log(0)
eps = 1e-15
avg_tapping_psd_db = 10 * np.log10(tapping_psd_avg_avg + eps)
avg_resting_psd_db = 10 * np.log10(resting_psd_avg_avg + eps)

# Step 6: Plot the results
plt.figure(figsize=(8, 6))
plt.plot(tapping_freqs, avg_tapping_psd_db, label='Tapping (dB)', color='b', linewidth=2)
plt.plot(resting_freqs, avg_resting_psd_db, label='Resting (dB)', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

# Add titles and labels
plt.title('Comparison of Tapping and Resting Power Spectra (in dB)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power (dB)')
plt.legend()
plt.tight_layout()
plt.show()


In [38]:
# Normalize based on the total power across all frequencies for each epoch

total_power_tapping = np.sum(tapping_psd_avg, axis=1)  # Sum over the frequency axis for each epoch
total_power_resting = np.sum(resting_psd_avg, axis=1)

# Step 2: Normalize the power in each frequency bin by the total power for each epoch
normalized_tapping_psd = tapping_psd_avg / total_power_tapping[:, np.newaxis]  # Broadcasting
normalized_resting_psd = resting_psd_avg / total_power_resting[:, np.newaxis]  # Broadcasting

# Step 3: Now `normalized_tapping_psd` and `normalized_resting_psd` contain the relative power

# Optionally, average across epochs to get a single power spectrum for each condition
avg_normalized_tapping_psd = np.mean(normalized_tapping_psd, axis=0)
avg_normalized_resting_psd = np.mean(normalized_resting_psd, axis=0)

# Plot the normalized power spectra
plt.figure(figsize=(8, 6))
plt.plot(tapping_freqs, avg_normalized_tapping_psd, label='Tapping (Normalized)', color='b', linewidth=2)
plt.plot(resting_freqs, avg_normalized_resting_psd, label='Resting (Normalized)', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

plt.title('Normalized (Frequency Band Normalization) Power Spectra Comparison: Tapping vs Resting')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Normalized Power')
plt.legend()
plt.tight_layout()
plt.show()


In [36]:
# Plot in dB unnormalized (log scale)

# Step 5: Convert power to dB (log scale)
# Add a small constant to avoid log(0)
eps = 1e-15
avg_freq_norm_tapping_psd_db = 10 * np.log10(avg_normalized_tapping_psd + eps)
avg_freq_norm_resting_psd_db = 10 * np.log10(avg_normalized_resting_psd + eps)

# Step 6: Plot the results
plt.figure(figsize=(8, 6))
plt.plot(tapping_freqs, avg_freq_norm_tapping_psd_db, label='Tapping (dB)', color='b', linewidth=2)
plt.plot(resting_freqs, avg_freq_norm_resting_psd_db, label='Resting (dB)', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

# Add titles and labels
plt.title('Comparison of Tapping and Resting Power Spectra (in dB)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power (dB)')
plt.legend()
plt.tight_layout()
plt.show()


In [37]:
# Normalize the PSDs to have the same scaling (e.g., z-score normalization)
resting_avg_psd_z_normalized = (resting_psd_avg_avg - resting_psd_avg_avg.mean()) / resting_psd_avg_avg.std()
tapping_avg_psd_z_normalized = (tapping_psd_avg_avg - tapping_psd_avg_avg.mean()) / tapping_psd_avg_avg.std()

# Plot the normalized power spectra
plt.figure(figsize=(8, 6))
plt.plot(tapping_freqs, tapping_avg_psd_z_normalized, label='Tapping (Normalized)', color='b', linewidth=2)
plt.plot(resting_freqs, resting_avg_psd_z_normalized, label='Resting (Normalized)', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

plt.title('Normalized (z-score normalization) Power Spectra Comparison: Tapping vs Resting')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Normalized Power')
plt.legend()
plt.tight_layout()
plt.show()


In [43]:
# Plot in dB unnormalized (log scale)

# Step 5: Convert power to dB (log scale)
# Add a small constant to avoid log(0)
eps = 1e-6
avg_z_norm_tapping_psd_db = 10 * np.log10(tapping_avg_psd_z_normalized + eps)
avg_z_norm_resting_psd_db = 10 * np.log10(resting_avg_psd_z_normalized + eps)
print(resting_avg_psd_z_normalized)

print(avg_z_norm_tapping_psd_db)
# Step 6: Plot the results
plt.figure(figsize=(8, 6))
plt.plot(tapping_freqs, avg_z_norm_tapping_psd_db, label='Tapping (dB)', color='b', linewidth=2)
plt.plot(resting_freqs, avg_z_norm_resting_psd_db, label='Resting (dB)', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

plt.title('Normalized (z-score normalization) Power Spectra Comparison: Tapping vs Resting')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Normalized Power')
plt.legend()
plt.tight_layout()
plt.show()


NameError: name 'tapping_avg_psd_z_normalized' is not defined

In [97]:
# Unnormalized plot of rest vs. tapping
print("Resting Frequencies:", resting_freqs)
print("Tapping Frequencies:", tapping_freqs)

print("Resting avg PSD shape:", resting_avg_psd.shape)
print("Tapping avg PSD shape:", tapping_avg_psd.shape)

# Plot the tapping power spectrum
plt.figure(figsize=(8, 6))
plt.plot(tapping_freqs, tapping_avg_psd.mean(axis=0), label='Tapping', color='b', linewidth=2)

# Plot the resting power spectrum
plt.plot(resting_freqs, resting_avg_psd.mean(axis=0), label='Resting', color='r', linestyle='--', linewidth=2)

# Add titles and labels
plt.title('Comparison of Tapping and Resting Power Spectra')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


Resting Frequencies: [ 3.90625  7.8125  11.71875 15.625   19.53125 23.4375  27.34375 31.25
 35.15625 39.0625  42.96875 46.875  ]
Tapping Frequencies: [ 3.90625  7.8125  11.71875 15.625   19.53125 23.4375  27.34375 31.25
 35.15625 39.0625  42.96875 46.875  ]
Resting avg PSD shape: (30, 12)
Tapping avg PSD shape: (184, 12)


In [41]:
# List of motor cortex electrodes (C3, C4, Cz for 10-20 system)
motor_cortex_channels = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'Cz']

# 1. Select the motor cortex electrodes for tapping and resting epochs
# Extract motor cortex data for tapping epochs
motor_cortex_tapping_epochs = group_tapping_epochs.copy()
motor_cortex_tapping_epochs = motor_cortex_tapping_epochs.load_data().pick(motor_cortex_channels)

# Extract motor cortex data for resting epochs
motor_cortex_resting_epochs = group_resting_epochs.copy()
motor_cortex_resting_epochs = motor_cortex_resting_epochs.load_data().pick(motor_cortex_channels)

# 2. Compute the power spectrum for the tapping state
sfreq = group_resting_epochs.info['sfreq']  # Sample frequency

# For tapping
motor_cortex_tapping_psd, motor_cortex_tapping_freqs = psd_array_welch(
    motor_cortex_tapping_epochs.get_data(), sfreq=sfreq, fmin=0.5, fmax=50, n_per_seg=256
)

# For resting
motor_cortex_resting_psd, motor_cortex_resting_freqs = psd_array_welch(
    motor_cortex_resting_epochs.get_data(), sfreq=sfreq, fmin=0.5, fmax=50, n_per_seg=256
)

# 3. Average the PSD across epochs and then across channels (focus on motor cortex)
motor_cortex_tapping_avg_psd = motor_cortex_tapping_psd.mean(axis=(0, 1))  # Averaging across epochs and channels
motor_cortex_resting_avg_psd = motor_cortex_resting_psd.mean(axis=(0, 1))  # Averaging across epochs and channels

# 4. Plot the comparison between tapping and resting power spectra
plt.figure(figsize=(10, 6))

# Plot Tapping power spectrum
plt.plot(motor_cortex_tapping_freqs, motor_cortex_tapping_avg_psd, label='Tapping', color='b', linewidth=2)

# Plot Resting power spectrum
plt.plot(motor_cortex_resting_freqs, motor_cortex_resting_avg_psd, label='Resting', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

# Add titles and labels
plt.title('Comparison of Tapping vs Resting Power Spectrum (Motor Cortex)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


In [37]:
# Define beta band range (13–20 Hz)
beta_band = (13, 20)
sfreq = group_resting_epochs.info['sfreq']

# Calculate PSDs for each participant separately
resting_beta_power = []
tapping_beta_power = []

for rest_epochs, tap_epochs in zip(resting_epochs_list, tapping_epochs_list):
    rest_psd, _ = psd_array_welch(
        rest_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
    )
    tap_psd, _ = psd_array_welch(
        tap_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
    )

    # Average power across the beta band frequencies
    resting_beta_power.append(rest_psd.mean(axis=2).mean(axis=0))  # Average across epochs
    tapping_beta_power.append(tap_psd.mean(axis=2).mean(axis=0))  # Average across epochs

# Convert lists to arrays for plotting
resting_beta_power = np.array(resting_beta_power)  # Shape: (n_participants, n_channels)
tapping_beta_power = np.array(tapping_beta_power)  # Shape: (n_participants, n_channels)

print(resting_beta_power.shape)

avg_resting_beta_power = resting_beta_power.mean(axis=1)  # Shape: (n_participants,)
avg_tapping_beta_power = tapping_beta_power.mean(axis=1)  # Shape: (n_participants,)


Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
(2, 58)


In [52]:
from mne.viz import plot_topomap

# Define beta band range (13–30 Hz)
beta_band = (13, 20)


# Update the raw data Info object
sfreq = group_resting_epochs.info['sfreq']
resting_psd, freqs = psd_array_welch(
    group_resting_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)
tapping_psd, _ = psd_array_welch(
    group_tapping_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)

print(resting_psd.shape)

resting_psd_avg_topo = resting_psd.mean(axis=2)
tapping_psd_avg_topo = tapping_psd.mean(axis=2)

print(resting_psd_avg_topo.shape)


total_power_tapping = np.sum(tapping_psd_avg_topo, axis=1)  # Sum over the frequency axis for each epoch
total_power_resting = np.sum(resting_psd_avg_topo, axis=1)

# Step 2: Normalize the power in each frequency bin by the total power for each epoch
normalized_tapping_psd = tapping_psd_avg / total_power_tapping[:, np.newaxis]  # Broadcasting
normalized_resting_psd = resting_psd_avg / total_power_resting[:, np.newaxis]  # Broadcasting

# Average beta power across epochs for each channel
avg_resting_beta_power = normalized_resting_psd.mean(axis=0)  # Shape: (n_channels,)
avg_tapping_beta_power = normalized_tapping_psd.mean(axis=0)  # Shape: (n_channels,)

print(avg_resting_beta_power.shape)
# Get the channel locations (from the raw object)
info = group_resting_epochs.info

# Calculate the difference in beta power (Tapping - Resting)
beta_power_difference = avg_tapping_beta_power - avg_resting_beta_power

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot Resting Beta Power
im1, _ = plot_topomap(avg_resting_beta_power, info, axes=axes[0], cmap='viridis', show=False)
axes[0].set_title('Resting Beta Power')

# Plot Tapping Beta Power
im2, _ = plot_topomap(avg_tapping_beta_power, info, axes=axes[1], cmap='viridis', show=False)
axes[1].set_title('Tapping Beta Power')

# Plot Difference in Beta Power
im3, _ = plot_topomap(beta_power_difference, info, axes=axes[2], cmap='RdBu_r', show=False)  # RdBu_r for diverging colormap
axes[2].set_title('Difference (Tapping - Resting)')

# Add a shared color bar
cbar = fig.colorbar(im3, ax=axes, orientation='horizontal', shrink=0.7)
cbar.set_label('Beta Power')

plt.tight_layout()
plt.show()



Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
(90, 57, 2)
(90, 57)


ValueError: operands could not be broadcast together with shapes (162,12) (346,1) 

In [86]:
# Normalized TopoMaps

import numpy as np
from mne.time_frequency import psd_array_welch
from mne.viz import plot_topomap
import matplotlib.pyplot as plt

# Define beta band range (13–20 Hz)
beta_band = (13, 20)

# Function to compute and normalize power for each participant
def compute_normalized_beta_power(epochs_list, sfreq, beta_band):
    normalized_beta_powers = []
    for epochs in epochs_list:
        # Compute power spectral density (PSD) for the current participant
        psd, _ = psd_array_welch(epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256)
        beta_power = psd.mean(axis=2)  # Average over frequency band (shape: n_epochs, n_channels)

        # Average across epochs to get power per channel
        avg_beta_power = beta_power.mean(axis=0)  # Shape: (n_channels,)

        # Normalize using Z-score (per participant)
        norm_beta_power = (avg_beta_power - avg_beta_power.mean()) / avg_beta_power.std()

        normalized_beta_powers.append(norm_beta_power)
    
    return np.array(normalized_beta_powers)  # Shape: (n_participants, n_channels)

# Compute normalized beta power for resting and tapping
sfreq = group_resting_epochs.info['sfreq']  # Sampling frequency
norm_resting_beta_power = compute_normalized_beta_power(resting_epochs_list, sfreq, beta_band)
norm_tapping_beta_power = compute_normalized_beta_power(tapping_epochs_list, sfreq, beta_band)

# Average normalized power across participants
mean_norm_resting_beta_power = norm_resting_beta_power.mean(axis=0)  # Shape: (n_channels,)
mean_norm_tapping_beta_power = norm_tapping_beta_power.mean(axis=0)  # Shape: (n_channels,)

# Compute difference in normalized beta power (Tapping - Resting)
mean_norm_beta_power_difference = mean_norm_tapping_beta_power - mean_norm_resting_beta_power

# Plot Topomaps
info = group_resting_epochs.info  # Use the info object from the epochs
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Plot Normalized Resting Beta Power
im1, _ = plot_topomap(mean_norm_resting_beta_power, info, axes=axes[0], cmap='viridis', show=False)
axes[0].set_title('Normalized Resting Beta Power')

# Plot Normalized Tapping Beta Power
im2, _ = plot_topomap(mean_norm_tapping_beta_power, info, axes=axes[1], cmap='viridis', show=False)
axes[1].set_title('Normalized Tapping Beta Power')

# Plot Difference in Normalized Beta Power
im3, _ = plot_topomap(mean_norm_beta_power_difference, info, axes=axes[2], cmap='RdBu_r', show=False)
axes[2].set_title('Normalized Difference (Tapping - Resting)')

# Add shared colorbar
cbar = fig.colorbar(im3, ax=axes, orientation='vertical', shrink=0.3)
cbar.set_label('Normalized Beta Power (Z-Score)')

plt.tight_layout()
plt.show()


Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\4004109724.py:61: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [70]:
# Define mu band range (8–13 Hz)
mu_band = (8, 13)

# Update the raw data Info object
sfreq = group_resting_epochs.info['sfreq']
resting_mu_psd, resting_freqs = psd_array_welch(
    group_resting_epochs.get_data(), sfreq=sfreq, fmin=mu_band[0], fmax=mu_band[1], n_per_seg=256
)
tapping_mu_psd, _ = psd_array_welch(
    group_tapping_epochs.get_data(), sfreq=sfreq, fmin=mu_band[0], fmax=mu_band[1], n_per_seg=256
)

# Average power across the beta band frequencies
resting_mu_power = resting_mu_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)
tapping_mu_power = tapping_mu_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)

# Average beta power across epochs for each channel
avg_resting_mu_power = resting_mu_power.mean(axis=0)  # Shape: (n_channels,)
avg_tapping_mu_power = tapping_mu_power.mean(axis=0)  # Shape: (n_channels,)

# Get the channel locations (from the raw object)
info = group_tapping_epochs.info

# Calculate the difference in beta power (Tapping - Resting)
mu_power_difference = avg_tapping_mu_power - avg_resting_mu_power

# Create subplots
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Plot Resting Beta Power
im1, _ = plot_topomap(avg_resting_mu_power, info, axes=axes[0], cmap='viridis', show=False)
axes[0].set_title('Resting Mu Power')

# Plot Tapping Beta Power
im2, _ = plot_topomap(avg_tapping_mu_power, info, axes=axes[1], cmap='viridis', show=False)
axes[1].set_title('Tapping Mu Power')

# Plot Difference in Beta Power
im3, _ = plot_topomap(mu_power_difference, info, axes=axes[2], cmap='RdBu_r', show=False)  # RdBu_r for diverging colormap
axes[2].set_title('Difference (Tapping - Resting)')

# Add a shared color bar
cbar = fig.colorbar(im3, ax=axes, orientation='vertical', shrink=0.7)
cbar.set_label('Mu Power')

plt.tight_layout()
plt.show()



Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\454394845.py:46: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [16]:
# Assuming avg_resting_beta_power and avg_tapping_beta_power are arrays of shape (n_participants,)
# If they aren't, ensure to average them properly.

# Create a list of participant labels
participants = [f"Person {i+1}" for i in range(len(avg_resting_beta_power))]

# Stack data for boxplot
data = [avg_resting_beta_power, avg_tapping_beta_power]

# Create boxplot
plt.figure(figsize=(8, 6))
box = plt.boxplot(data, labels=['Resting', 'Tapping'], patch_artist=True)

# Add custom colors for the boxplot (optional)
colors = ['lightblue', 'lightgreen']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Plot individual participant data with lines
for i, (rest, tap) in enumerate(zip(avg_resting_beta_power, avg_tapping_beta_power)):
    plt.plot([1, 2], [rest, tap], marker='o', label=participants[i])

# Annotate and customize
plt.title("Comparison of Beta Band Power (13–20 Hz) in Resting and Tapping Conditions")
plt.ylabel("Average Beta Power")
plt.xlabel("Condition")
plt.legend(loc='best', title='Participants', fontsize='small', bbox_to_anchor=(1.05, 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show plot
plt.tight_layout()
plt.show()


C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\2284367212.py:12: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  box = plt.boxplot(data, labels=['Resting', 'Tapping'], patch_artist=True)


### Get Data for Delay vs. No Delay Analysis (Condition Check)

In [72]:
# Define your files
delay_files = ["participant_01_delay-epo.fif", "participant_02_delay-epo.fif", "participant_03_delay-epo.fif"]
no_delay_files = ["participant_01_no_delay-epo.fif", "participant_02_no_delay-epo.fif", "participant_03_no_delay-epo.fif"]

# Define channels to filter out
exclude_channels = ['BIP3', 'BIP7', 'BIP8', 'BIP9']

# Load epochs and harmonize channels
def harmonize_epochs(file_list, exclude_channels):
    epochs_list = []
    common_channels = None  # To store the intersection of channel names

    # Step 1: Load all epochs and find common channels
    for file in file_list:
        epochs = mne.read_epochs(file, preload=True)
        if common_channels is None:
            common_channels = set(epochs.info['ch_names'])
        else:
            common_channels &= set(epochs.info['ch_names'])
        epochs_list.append(epochs)

    # Retain only common channels
    common_channels = list(common_channels)
    print(f"Common channels across all files: {common_channels}")

    # Step 2: Drop excluded channels from common channels
    common_channels = [ch for ch in common_channels if ch not in exclude_channels]
    print(f"Channels after exclusion: {common_channels}")

    # Step 3: Apply common channel set to each epochs object
    print(common_channels)

    harmonized_epochs_list = []
    for epochs in epochs_list:
        epochs.pick_channels(common_channels)
        harmonized_epochs_list.append(epochs)

    return harmonized_epochs_list

# Process the files
delay_epochs_list = harmonize_epochs(delay_files, exclude_channels)
no_delay_epochs_list = harmonize_epochs(no_delay_files, exclude_channels)

# Combine epochs
group_delay_epochs = mne.concatenate_epochs(delay_epochs_list)
group_no_delay_epochs = mne.concatenate_epochs(no_delay_epochs_list)

print(f"Combined resting epochs shape: {group_delay_epochs.get_data().shape}")
print(f"Combined tapping epochs shape: {group_no_delay_epochs.get_data().shape}")


Reading c:\Users\ymijs\OneDrive\Documents\Internship\Apathy-Agency\EEG Experiment\Data Analysis\participant_01_delay-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    9000.00 ms
        0 CTF compensation matrices available
Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\ymijs\OneDrive\Documents\Internship\Apathy-Agency\EEG Experiment\Data Analysis\participant_02_delay-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    9000.00 ms
        0 CTF compensation matrices available
Not setting metadata
58 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\ymijs\OneDrive\Documents\Internship\Apathy-Agency\EEG Experiment\Data Analysis\participant_03_delay-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    9000.00 ms
        0 CTF compensation matrices available
Not setting metadata
50 matching events found

C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\2550560006.py:45: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  group_delay_epochs = mne.concatenate_epochs(delay_epochs_list)


Not setting metadata
114 matching events found
No baseline correction applied


C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\2550560006.py:46: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  group_no_delay_epochs = mne.concatenate_epochs(no_delay_epochs_list)


Not setting metadata
232 matching events found
No baseline correction applied
Combined resting epochs shape: (114, 57, 10001)
Combined tapping epochs shape: (232, 57, 10001)


In [89]:
import numpy as np
import matplotlib.pyplot as plt
from mne.time_frequency import psd_array_welch

# Define theta band range (4–8 Hz)
theta_band = (4, 8)
sfreq = group_delay_epochs.info['sfreq']

# Initialize lists to store normalized theta power for each participant
normalized_delay_theta_power = []
normalized_no_delay_theta_power = []

# Compute and normalize theta power for each participant
for delay_epochs, no_delay_epochs in zip(delay_epochs_list, no_delay_epochs_list):
    # Calculate PSDs for delay and no-delay conditions
    delay_psd, _ = psd_array_welch(
        delay_epochs.get_data(), sfreq=sfreq, fmin=theta_band[0], fmax=theta_band[1], n_per_seg=256
    )
    no_delay_psd, _ = psd_array_welch(
        no_delay_epochs.get_data(), sfreq=sfreq, fmin=theta_band[0], fmax=theta_band[1], n_per_seg=256
    )

    # Average power across the theta band frequencies and epochs (across channels)
    delay_theta_power = delay_psd.mean(axis=2).mean(axis=0)  # Shape: (n_channels,)
    no_delay_theta_power = no_delay_psd.mean(axis=2).mean(axis=0)  # Shape: (n_channels,)

    # Normalize per participant (Z-score normalization across channels)
    delay_theta_power_norm = (delay_theta_power - delay_theta_power.mean()) / delay_theta_power.std()
    no_delay_theta_power_norm = (no_delay_theta_power - no_delay_theta_power.mean()) / no_delay_theta_power.std()

    # Store the normalized values (mean across channels)
    normalized_delay_theta_power.append(delay_theta_power_norm.mean())
    normalized_no_delay_theta_power.append(no_delay_theta_power_norm.mean())

# Convert lists to arrays for plotting
normalized_delay_theta_power = np.array(normalized_delay_theta_power)  # Shape: (n_participants,)
normalized_no_delay_theta_power = np.array(normalized_no_delay_theta_power)  # Shape: (n_participants,)

# Create boxplot
plt.figure(figsize=(8, 6))
box = plt.boxplot([normalized_delay_theta_power, normalized_no_delay_theta_power], 
                  labels=['Delay', 'No Delay'], patch_artist=True)

# Add custom colors for the boxplot (optional)
colors = ['lightblue', 'lightgreen']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Plot individual participant data with lines
for i, (delay, no_delay) in enumerate(zip(normalized_delay_theta_power, normalized_no_delay_theta_power)):
    plt.plot([1, 2], [delay, no_delay], marker='o', label=f'Participant {i+1}' if i == 0 else "")  # Label only first participant

# Annotate and customize
plt.title("Comparison of Theta Band Power (4 - 8 Hz) in Delay and No Delay Conditions")
plt.ylabel("Normalized Theta Power (Z-score)")
plt.xlabel("Condition")
plt.legend(loc='best', title='Participants', fontsize='small', bbox_to_anchor=(1.05, 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show plot
plt.tight_layout()
plt.show()


Effective window size : 0.256 (s)


Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\2786179417.py:41: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  box = plt.boxplot([normalized_delay_theta_power, normalized_no_delay_theta_power],


In [75]:
# Assuming avg_resting_beta_power and avg_tapping_beta_power are arrays of shape (n_participants,)
# If they aren't, ensure to average them properly.

# Create a list of participant labels
participants = [f"Person {i+1}" for i in range(len(avg_delay_beta_power))]

# Stack data for boxplot
data = [avg_delay_beta_power, avg_no_delay_beta_power]

# Create boxplot
plt.figure(figsize=(8, 6))
box = plt.boxplot(data, labels=['Delay', 'No Delay'], patch_artist=True)

# Add custom colors for the boxplot (optional)
colors = ['lightblue', 'lightgreen']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Plot individual participant data with lines
for i, (rest, tap) in enumerate(zip(avg_delay_beta_power, avg_no_delay_beta_power)):
    plt.plot([1, 2], [rest, tap], marker='o', label=participants[i])

# Annotate and customize
plt.title("Comparison of Theta Band Power (4 - 8 Hz) in Delay and No Delay Conditions")
plt.ylabel("Average Theta Power")
plt.xlabel("Condition")
plt.legend(loc='best', title='Participants', fontsize='small', bbox_to_anchor=(1.05, 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show plot
plt.tight_layout()
plt.show()


C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\2818549051.py:12: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  box = plt.boxplot(data, labels=['Delay', 'No Delay'], patch_artist=True)


In [40]:
from mne.viz import plot_topomap

# Define beta band range (13–30 Hz)
beta_band = (4, 8)


# Update the raw data Info object
sfreq = group_delay_epochs.info['sfreq']
delay_psd, freqs = psd_array_welch(
    group_delay_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)
no_delay_psd, _ = psd_array_welch(
    group_no_delay_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)

# Average power across the beta band frequencies
delay_beta_power = delay_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)
no_delay_beta_power = no_delay_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)

# Average beta power across epochs for each channel
avg_delay_beta_power = delay_beta_power.mean(axis=0)  # Shape: (n_channels,)
avg_no_delay_beta_power = no_delay_beta_power.mean(axis=0)  # Shape: (n_channels,)

# Get the channel locations (from the raw object)
info = group_delay_epochs.info

# Calculate the difference in beta power (Tapping - Resting)
beta_power_difference = avg_delay_beta_power - avg_no_delay_beta_power

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot Resting Beta Power
im1, _ = plot_topomap(avg_delay_beta_power, info, axes=axes[0], cmap='viridis', show=False)
axes[0].set_title('Delay Theta Power')

# Plot Tapping Beta Power
im2, _ = plot_topomap(avg_no_delay_beta_power, info, axes=axes[1], cmap='viridis', show=False)
axes[1].set_title('No Delay Theta Power')

# Plot Difference in Beta Power
im3, _ = plot_topomap(beta_power_difference, info, axes=axes[2], cmap='RdBu_r', show=False)  # RdBu_r for diverging colormap
axes[2].set_title('Difference (Delay - No Delay)')

# Add a shared color bar
cbar = fig.colorbar(im3, ax=axes, orientation='horizontal', shrink=0.7)
cbar.set_label('Theta Power')

plt.tight_layout()
plt.show()



Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


C:\Users\ymijs\AppData\Local\Temp\ipykernel_27380\3136495991.py:49: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [136]:
# Now we create the power spectrum graph for the no delay state
sfreq = raw.info['sfreq']
no_delay_psd, no_delay_freqs  = psd_array_welch(
    no_delay_tapping_epochs.get_data(), sfreq=sfreq, fmin=0.5, fmax=50, n_per_seg=256
)
print(no_delay_psd.shape)

# Average the PSD across all channels
no_delay_avg_psd = no_delay_psd.mean(axis=1)  # Shape: (n_freqs, n_epochs)
no_delay_avg_avg_psd = no_delay_avg_psd.mean(axis=0)  # Shape: (n_freqs, n_epochs)


# Now we create the power spectrum graph for the delay state
sfreq = raw.info['sfreq']
delay_psd, delay_freqs  = psd_array_welch(
    delay_tapping_epochs.get_data(), sfreq=sfreq, fmin=0.5, fmax=50, n_per_seg=256
)
print(delay_psd.shape)

# Average the PSD across all channels
delay_avg_psd = delay_psd.mean(axis=1)  # Shape: (n_freqs, n_epochs)
delay_avg_avg_psd = delay_avg_psd.mean(axis=0)  # Shape: (n_freqs, n_epochs)


Effective window size : 0.256 (s)
(126, 61, 12)
Effective window size : 0.256 (s)
(58, 61, 12)


In [137]:
# Normalize based on the total power across all frequencies for each epoch

total_power_no_delay = np.sum(no_delay_avg_psd, axis=1)  # Sum over the frequency axis for each epoch
total_power_delay = np.sum(delay_avg_psd, axis=1)

# Step 2: Normalize the power in each frequency bin by the total power for each epoch
normalized_no_delay_psd = no_delay_avg_psd / total_power_no_delay[:, np.newaxis]  # Broadcasting
normalized_delay_psd = delay_avg_psd / total_power_delay[:, np.newaxis]  # Broadcasting

# Step 3: Now `normalized_tapping_psd` and `normalized_resting_psd` contain the relative power

# Optionally, average across epochs to get a single power spectrum for each condition
avg_normalized_no_delay_psd = np.mean(normalized_no_delay_psd, axis=0)
avg_normalized_delay_psd = np.mean(normalized_delay_psd, axis=0)

# Plot the normalized power spectra
plt.figure(figsize=(8, 6))
plt.plot(no_delay_freqs, avg_normalized_no_delay_psd, label='No Delay (Normalized)', color='b', linewidth=2)
plt.plot(delay_freqs, avg_normalized_delay_psd, label='Delay (Normalized)', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

# Highlight Theta range (8-13 Hz)
plt.axvspan(4, 8, color='pink', alpha=0.3, label='Theta Range (4-8 Hz)')

plt.title('Normalized (Frequency Band Normalization) Power Spectra Comparison: Tapping vs Resting')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Normalized Power')
plt.legend()
plt.tight_layout()
plt.show()


In [139]:
# Define beta band range (13–30 Hz)
beta_band = (13, 30)


# Update the raw data Info object
sfreq = raw.info['sfreq']
no_delay_beta_psd, no_delay_beta_freqs = psd_array_welch(
    no_delay_tapping_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)
delay_beta_psd, _ = psd_array_welch(
    delay_tapping_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)

print(resting_epochs.info['ch_names'])

# Average power across the beta band frequencies
no_delay_beta_power = no_delay_beta_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)
delay_beta_power = delay_beta_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)

# Average beta power across epochs for each channel
avg_no_delay_beta_power = no_delay_beta_power.mean(axis=0)  # Shape: (n_channels,)
avg_delay_beta_power = delay_beta_power.mean(axis=0)  # Shape: (n_channels,)

from mne.viz import plot_topomap

# Get the channel locations (from the raw object)
print(no_delay_tapping_epochs.info)
info = no_delay_tapping_epochs.info

# Calculate the difference in beta power (Tapping - Resting)
beta_power_difference = avg_no_delay_beta_power - avg_delay_beta_power

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot Resting Beta Power
im1, _ = plot_topomap(avg_no_delay_beta_power, info, axes=axes[0], cmap='viridis', show=False)
axes[0].set_title('No Delay Beta Power')

# Plot Tapping Beta Power
im2, _ = plot_topomap(avg_delay_beta_power, info, axes=axes[1], cmap='viridis', show=False)
axes[1].set_title('Tapping Beta Power')

# Plot Difference in Beta Power
im3, _ = plot_topomap(beta_power_difference, info, axes=axes[2], cmap='RdBu_r', show=False)  # RdBu_r for diverging colormap
axes[2].set_title('Difference (No Delay - Delay)')

# Add a shared color bar
cbar = fig.colorbar(im3, ax=axes, orientation='horizontal', shrink=0.7)
cbar.set_label('Beta Power')

plt.tight_layout()
plt.show()



Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CP4', 'P5', 'P1', 'P2', 'P6', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8']
<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, M1, T7, ...
 chs: 61 EEG
 custom_ref_applied: True
 dig: 64 items (3 Cardinal, 61 EEG)
 highpass: 0.5 Hz
 lowpass: 50.0 Hz
 meas_date: 2024-11-22 14:06:42 UTC
 nchan: 61
 projs: []
 sfreq: 1000.0 Hz
>


C:\Users\ymijs\AppData\Local\Temp\ipykernel_18376\3145299124.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [129]:
# Define beta band range (13–30 Hz)
beta_band = (4, 8)


# Update the raw data Info object
sfreq = raw.info['sfreq']
no_delay_beta_psd, no_delay_beta_freqs = psd_array_welch(
    no_delay_tapping_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)
delay_beta_psd, _ = psd_array_welch(
    delay_tapping_epochs.get_data(), sfreq=sfreq, fmin=beta_band[0], fmax=beta_band[1], n_per_seg=256
)

print(resting_epochs.info['ch_names'])

# Average power across the beta band frequencies
no_delay_beta_power = no_delay_beta_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)
delay_beta_power = delay_beta_psd.mean(axis=2)  # Shape: (n_epochs, n_channels)

# Average beta power across epochs for each channel
avg_no_delay_beta_power = no_delay_beta_power.mean(axis=0)  # Shape: (n_channels,)
avg_delay_beta_power = delay_beta_power.mean(axis=0)  # Shape: (n_channels,)

from mne.viz import plot_topomap

# Get the channel locations (from the raw object)
print(no_delay_tapping_epochs.info)
info = no_delay_tapping_epochs.info

# Calculate the difference in beta power (Tapping - Resting)
beta_power_difference = avg_no_delay_beta_power - avg_delay_beta_power

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot Resting Beta Power
im1, _ = plot_topomap(avg_no_delay_beta_power, info, axes=axes[0], cmap='viridis', show=False)
axes[0].set_title('No Delay Beta Power')

# Plot Tapping Beta Power
im2, _ = plot_topomap(avg_delay_beta_power, info, axes=axes[1], cmap='viridis', show=False)
axes[1].set_title('Tapping Beta Power')

# Plot Difference in Beta Power
im3, _ = plot_topomap(beta_power_difference, info, axes=axes[2], cmap='RdBu_r', show=False)  # RdBu_r for diverging colormap
axes[2].set_title('Difference (No Delay - Delay)')

# Add a shared color bar
cbar = fig.colorbar(im3, ax=axes, orientation='horizontal', shrink=0.7)
cbar.set_label('Beta Power')

plt.tight_layout()
plt.show()



Effective window size : 0.256 (s)
Effective window size : 0.256 (s)
['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CP4', 'P5', 'P1', 'P2', 'P6', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8']
<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, M1, T7, ...
 chs: 61 EEG
 custom_ref_applied: True
 dig: 64 items (3 Cardinal, 61 EEG)
 highpass: 0.5 Hz
 lowpass: 50.0 Hz
 meas_date: 2024-11-22 14:06:42 UTC
 nchan: 61
 projs: []
 sfreq: 1000.0 Hz
>


C:\Users\ymijs\AppData\Local\Temp\ipykernel_18376\2272899070.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [138]:
# List of motor cortex electrodes (C3, C4, Cz for 10-20 system)
motor_cortex_channels = ['C3', 'C4', 'Cz']

# 1. Select the motor cortex electrodes for tapping and resting epochs
# Extract motor cortex data for tapping epochs
motor_cortex_no_delay_epochs = no_delay_tapping_epochs.copy()
motor_cortex_no_delay_epochs = motor_cortex_no_delay_epochs.load_data().pick(motor_cortex_channels)

# Extract motor cortex data for resting epochs
motor_cortex_delay_epochs = delay_tapping_epochs.copy()
motor_cortex_delay_epochs = motor_cortex_delay_epochs.load_data().pick(motor_cortex_channels)

# 2. Compute the power spectrum for the tapping state
sfreq = raw.info['sfreq']  # Sample frequency

# For tapping
motor_cortex_no_delay_psd, motor_cortex_no_delay_freqs = psd_array_welch(
    motor_cortex_no_delay_epochs.get_data(), sfreq=sfreq, fmin=0.5, fmax=50, n_per_seg=256
)

# For resting
motor_cortex_delay_psd, motor_cortex_delay_freqs = psd_array_welch(
    motor_cortex_delay_epochs.get_data(), sfreq=sfreq, fmin=0.5, fmax=50, n_per_seg=256
)

# 3. Average the PSD across epochs and then across channels (focus on motor cortex)
motor_cortex_no_delay_avg_psd = motor_cortex_no_delay_psd.mean(axis=(0, 1))  # Averaging across epochs and channels
motor_cortex_delay_avg_psd = motor_cortex_delay_psd.mean(axis=(0, 1))  # Averaging across epochs and channels

# 4. Plot the comparison between tapping and resting power spectra
plt.figure(figsize=(10, 6))

# Plot Tapping power spectrum
plt.plot(motor_cortex_no_delay_freqs, motor_cortex_no_delay_avg_psd, label='No Delay', color='b', linewidth=2)

# Plot Resting power spectrum
plt.plot(motor_cortex_delay_freqs, motor_cortex_delay_avg_psd, label='Delay', color='r', linestyle='--', linewidth=2)

# Highlight Beta range (13-30 Hz)
plt.axvspan(13, 30, color='yellow', alpha=0.3, label='Beta Range (13-30 Hz)')

# Highlight Mu range (8-13 Hz)
plt.axvspan(8, 13, color='green', alpha=0.3, label='Mu Range (8-13 Hz)')

# Add titles and labels
plt.title('Comparison of Tapping vs Resting Power Spectrum (Motor Cortex)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


### Third Analysis comparing the decision point of accepts and rejects with the resting state

In [167]:
# Now divide the tapping task data up in accepted and declined tasks during the decision time

# First map each event ID to its relevant binary code (in int)
event_id_to_binary = {
    code: int(desc.split('/s')[-1]) for desc, code in tapping_events_id.items() if 'Stimulus/s' in desc
}

# Identify start and end of tapping events, marked by a 64 bit exclusion and a 1 bit on or off for start or end
def is_decision_start(binary_val):
    return (binary_val & 64 == 64) and (binary_val & 1 == 0)

def is_decision_end(binary_val):
    return (binary_val & 64 == 64) and (binary_val & 1 == 1)

def is_decision_accepted(binary_val):
    return (binary_val & 64 == 0)

# Store starting and ending events
decision_start_events = []
decision_end_events = []

for event in tapping_events:  # Iterate over all events
    event_code = event[2]
    if event_code in event_id_to_binary:  # Only process stimulus events
        binary_val = event_id_to_binary[event_code]
        if is_decision_start(binary_val):
            decision_start_events.append(event)
        elif is_decision_end(binary_val):
            decision_end_events.append(event)

print(len(decision_start_events))
print(len(decision_end_events))

# As there is a mismatch between starting and ending events we will look for end events that happen within 7.5 seconds after the start of the tapping event and pair the first end event in that timeframe with the starting event
def match_start_end_events(start_events, end_events, time_window=5):
    matched_events = []

    for start in start_events:
        start_time = start[0] / tapping_area.info['sfreq']  # Convert sample index to time in seconds
        # Search for the end event within 8 seconds after the start event
        for end in end_events:
            end_time = end[0] / raw.info['sfreq']  # Convert sample index to time in seconds
            if start_time < end_time <= (start_time + time_window) and event_id_to_binary[start[2]] + 1 == event_id_to_binary[end[2]]:
                matched_events.append((start, end))
                break  # Once we find the first matching end, we move to the next start event

    return matched_events

all_decision_pairs = match_start_end_events(decision_start_events, decision_end_events, time_window=5)
print(len(all_decision_pairs))
# Finally, we filter some of these tapping pairs, because of failed tests, when they last less than 7 seconds

# Calculate the duration of each tapping pair (end time - start time)
#tapping_durations = np.array([end[0] - start[0] for start, end in all_tapping_pairs])
# Filter out tapping pairs with a duration less than 7 seconds (assuming the sampling rate is known, e.g., 1000 Hz)
# If the sampling rate is 1000 Hz, 7 seconds would correspond to 7000 samples. 
# Adding a 200ms boundary because the timing of the triggers is not perfect and the there might be a ms-level difference between start and end that make it less than 7 seconds
#min_duration_samples = 7 * 1000 - 200 # adjust this based on your actual sampling rate
#valid_tapping_indices = tapping_durations >= min_duration_samples
# Now we filter the tapping pairs to include only the valid ones
#valid_tapping_pairs = np.array(all_tapping_pairs)[valid_tapping_indices]

print(tapping_events)

accepted_decision_pairs = []
declined_decision_pairs = []

def is_tapping(binary_val):
    return (binary_val & 64 == 0)

for first_event, second_event in all_decision_pairs:
    second_event_index = [i for i, event in enumerate(tapping_events) if event[0] == second_event[0]][0]
    print(second_event_index)
    if second_event_index is not None and second_event_index + 1 < len(tapping_events):
        # Check the event following the second event
        print(second_event_index)
        next_event = tapping_events[second_event_index + 1]
        print(second_event[0])
        print(next_event[2])
        if is_tapping(event_id_to_binary[next_event[2]]):
            accepted_decision_pairs.append((first_event, second_event))
        else:
            declined_decision_pairs.append((first_event, second_event))

print(len(accepted_decision_pairs))
print(len(declined_decision_pairs))


1081
819
825
[[1007191       0   10023]
 [1012306       0   10024]
 [1024382       0   10023]
 ...
 [6317589       0   10024]
 [6325349       0   10023]
 [6332351       0   10024]]
33
33
1391541
10042
40
40
1412630
10016
47
47
1432949
10032
55
55
1449222
10050
59
59
1467167
10042
63
63
1484373
10040
67
67
1503519
10077
75
75
1513559
10016
83
83
1536152
10069
85
85
1539448
10075
100
100
1550313
10002
113
113
1576258
10083
119
119
1580874
10040
123
123
1597530
10002
127
127
1614141
10032
131
131
1630195
10001
137
137
1633891
10016
144
144
1651916
10042
150
150
1669453
10040
163
163
1692605
10083
165
165
1693933
10050
169
169
1710894
10040
173
173
1727566
10048
177
177
1743367
10044
181
181
1759775
10071
183
183
1761007
10032
190
190
1778440
10050
200
200
1800400
10081
206
206
1805064
10001
204
204
1804120
10081
206
206
1805064
10001
209
209
1807056
10046
215
215
1826769
10077
229
229
1837129
10050
241
241
1858538
10044
252
252
1880178
10016
273
273
1908107
10002
277
277
1924372
10044
286